# Week 10: Advanced Marketing Mix Modeling in Production

## Learning Objectives
- Build production-scale Marketing Mix Models (MMM)
- Implement Bayesian MMM with PyMC3
- Design hierarchical models for geo/segment MMM
- Automate model selection and validation
- Build budget optimization systems with constraints
- Implement forecasting at scale
- Deploy enterprise MMM platforms
- Handle adstock and saturation effects

## Prerequisites
```bash
pip install pandas numpy scipy statsmodels
pip install pymc3 arviz theano
pip install scikit-learn xgboost lightgbm
pip install plotly seaborn matplotlib
pip install cvxpy scipy.optimize
pip install psycopg2-binary sqlalchemy
```

## 1. Setup and Environment

In [ ]:
# Standard library
import os
import sys
import logging
import time
import warnings
import pickle
import json
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union
import gc

# Data manipulation
import numpy as np
import pandas as pd

# Statistics and modeling
from scipy import stats, optimize
from scipy.special import expit
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pymc3 as pm
import arviz as az
import theano
import theano.tensor as tt

# Machine learning
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import xgboost as xgb
import lightgbm as lgb

# Optimization
import cvxpy as cp

# Database
import psycopg2
from sqlalchemy import create_engine

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

print(f"PyMC3: {pm.__version__}")
print(f"ArviZ: {az.__version__}")
print(f"NumPy: {np.__version__}")
print("Environment ready!")

## 2. Adstock and Saturation Transformations

In [ ]:
class AdstockTransformations:
    """Marketing transformations for adstock and saturation."""
    
    @staticmethod
    def geometric_adstock(x: np.ndarray, theta: float) -> np.ndarray:
        """Apply geometric adstock transformation.
        
        Args:
            x: Marketing spend array
            theta: Decay rate (0-1), higher = longer carryover
        """
        adstocked = np.zeros_like(x, dtype=float)
        adstocked[0] = x[0]
        
        for t in range(1, len(x)):
            adstocked[t] = x[t] + theta * adstocked[t-1]
        
        return adstocked
    
    @staticmethod
    def delayed_adstock(x: np.ndarray, theta: float, L: int) -> np.ndarray:
        """Apply delayed adstock with maximum length L.
        
        Args:
            x: Marketing spend array
            theta: Decay rate
            L: Maximum lag periods
        """
        weights = np.array([theta ** i for i in range(L)])
        weights = weights / weights.sum()
        
        adstocked = np.convolve(x, weights, mode='same')
        return adstocked
    
    @staticmethod
    def hill_saturation(x: np.ndarray, K: float, S: float) -> np.ndarray:
        """Apply Hill saturation curve.
        
        Args:
            x: Marketing spend (adstocked)
            K: Half-saturation point
            S: Shape parameter (> 0)
        """
        return x ** S / (K ** S + x ** S)
    
    @staticmethod
    def logistic_saturation(x: np.ndarray, k: float, x0: float) -> np.ndarray:
        """Apply logistic saturation.
        
        Args:
            x: Marketing spend (adstocked)
            k: Growth rate
            x0: Midpoint
        """
        return 1 / (1 + np.exp(-k * (x - x0)))
    
    @staticmethod
    def transform_channel(x: np.ndarray, 
                         adstock_theta: float,
                         saturation_K: float,
                         saturation_S: float) -> np.ndarray:
        """Apply full transformation pipeline to channel."""
        # Apply adstock
        adstocked = AdstockTransformations.geometric_adstock(x, adstock_theta)
        
        # Apply saturation
        saturated = AdstockTransformations.hill_saturation(adstocked, saturation_K, saturation_S)
        
        return saturated


# Example: Visualize transformations
def visualize_transformations():
    """Visualize adstock and saturation effects."""
    
    # Generate sample spend data
    spend = np.zeros(52)
    spend[10] = 100  # Single spike
    
    # Apply adstock
    adstock_low = AdstockTransformations.geometric_adstock(spend, 0.3)
    adstock_high = AdstockTransformations.geometric_adstock(spend, 0.7)
    
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=['Adstock Effect', 'Saturation Curves'])
    
    # Adstock plot
    fig.add_trace(go.Scatter(y=spend, name='Original', line=dict(dash='dot')), row=1, col=1)
    fig.add_trace(go.Scatter(y=adstock_low, name='Theta=0.3'), row=1, col=1)
    fig.add_trace(go.Scatter(y=adstock_high, name='Theta=0.7'), row=1, col=2)
    
    # Saturation plot
    x = np.linspace(0, 100, 1000)
    hill = AdstockTransformations.hill_saturation(x, K=50, S=2)
    fig.add_trace(go.Scatter(x=x, y=hill, name='Hill Saturation'), row=1, col=2)
    
    fig.update_layout(height=400, showlegend=True)
    return fig

# fig = visualize_transformations()
# fig.show()

print("Adstock transformations ready")

## 3. Bayesian MMM with PyMC3

In [ ]:
class BayesianMMM:
    """Bayesian Marketing Mix Model using PyMC3."""
    
    def __init__(self, channel_names: List[str]):
        self.channel_names = channel_names
        self.model = None
        self.trace = None
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def build_model(self, 
                   y: np.ndarray,
                   X_media: np.ndarray,
                   X_control: Optional[np.ndarray] = None) -> pm.Model:
        """Build Bayesian MMM model.
        
        Args:
            y: Target variable (sales, conversions, etc.)
            X_media: Media spend matrix (n_periods x n_channels)
            X_control: Control variables (seasonality, events, etc.)
        """
        
        n_channels = X_media.shape[1]
        n_periods = len(y)
        
        with pm.Model() as model:
            # Priors for adstock parameters (one per channel)
            theta = pm.Beta('theta', alpha=2, beta=2, shape=n_channels)
            
            # Priors for saturation parameters
            K = pm.HalfNormal('K', sigma=1, shape=n_channels)
            S = pm.HalfNormal('S', sigma=1, shape=n_channels)
            
            # Transform media variables
            media_adstocked = pm.Deterministic(
                'media_adstocked',
                tt.stack([self._geometric_adstock_theano(X_media[:, i], theta[i]) 
                         for i in range(n_channels)], axis=1)
            )
            
            media_transformed = pm.Deterministic(
                'media_transformed',
                media_adstocked ** S / (K ** S + media_adstocked ** S)
            )
            
            # Priors for coefficients
            beta_media = pm.HalfNormal('beta_media', sigma=1, shape=n_channels)
            
            # Baseline and trend
            intercept = pm.Normal('intercept', mu=y.mean(), sigma=y.std())
            trend = pm.Normal('trend', mu=0, sigma=0.1)
            
            # Control variables (if provided)
            if X_control is not None:
                beta_control = pm.Normal('beta_control', mu=0, sigma=1, 
                                        shape=X_control.shape[1])
                control_contrib = pm.math.dot(X_control, beta_control)
            else:
                control_contrib = 0
            
            # Linear combination
            time_trend = trend * np.arange(n_periods)
            media_contrib = pm.math.dot(media_transformed, beta_media)
            
            mu = intercept + time_trend + media_contrib + control_contrib
            
            # Likelihood
            sigma = pm.HalfNormal('sigma', sigma=y.std())
            likelihood = pm.Normal('y', mu=mu, sigma=sigma, observed=y)
        
        self.model = model
        return model
    
    def _geometric_adstock_theano(self, x, theta):
        """Geometric adstock in Theano for PyMC3."""
        
        def adstock_step(x_t, adstock_prev, theta):
            adstock_t = x_t + theta * adstock_prev
            return adstock_t
        
        adstock, _ = theano.scan(
            fn=adstock_step,
            sequences=x,
            outputs_info=tt.zeros(1),
            non_sequences=theta
        )
        
        return adstock.flatten()
    
    def fit(self, draws: int = 2000, tune: int = 1000, 
           chains: int = 2, target_accept: float = 0.9):
        """Fit the Bayesian MMM model."""
        
        self.logger.info("Starting MCMC sampling")
        
        with self.model:
            self.trace = pm.sample(
                draws=draws,
                tune=tune,
                chains=chains,
                target_accept=target_accept,
                return_inferencedata=True,
                cores=chains
            )
        
        self.logger.info("Sampling complete")
        return self.trace
    
    def get_channel_contributions(self) -> pd.DataFrame:
        """Calculate channel contributions to total sales."""
        
        # Extract posterior means
        beta_media = self.trace.posterior['beta_media'].mean(dim=['chain', 'draw']).values
        media_transformed = self.trace.posterior['media_transformed'].mean(dim=['chain', 'draw']).values
        
        # Calculate contributions
        contributions = media_transformed * beta_media
        
        df = pd.DataFrame(
            contributions,
            columns=self.channel_names
        )
        
        return df
    
    def get_roas(self, X_media: np.ndarray) -> pd.DataFrame:
        """Calculate Return on Ad Spend (ROAS) for each channel."""
        
        contributions = self.get_channel_contributions()
        total_contribution = contributions.sum(axis=0)
        total_spend = pd.Series(X_media.sum(axis=0), index=self.channel_names)
        
        roas = total_contribution / total_spend
        
        return pd.DataFrame({
            'channel': self.channel_names,
            'total_spend': total_spend.values,
            'total_contribution': total_contribution.values,
            'roas': roas.values
        }).sort_values('roas', ascending=False)
    
    def plot_diagnostics(self):
        """Plot model diagnostics."""
        
        # Trace plots
        az.plot_trace(self.trace, var_names=['beta_media', 'theta'])
        plt.tight_layout()
        plt.show()
        
        # Posterior plots
        az.plot_posterior(self.trace, var_names=['beta_media'], ref_val=0)
        plt.tight_layout()
        plt.show()
    
    def save_model(self, filepath: str):
        """Save trained model."""
        with open(filepath, 'wb') as f:
            pickle.dump({
                'trace': self.trace,
                'channel_names': self.channel_names
            }, f)
        self.logger.info(f"Model saved to {filepath}")


print("Bayesian MMM class ready")

## 4. Hierarchical MMM for Geo/Segment Modeling

In [ ]:
class HierarchicalMMM:
    """Hierarchical Bayesian MMM for multiple geos or segments."""
    
    def __init__(self, channel_names: List[str], geo_names: List[str]):
        self.channel_names = channel_names
        self.geo_names = geo_names
        self.model = None
        self.trace = None
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def build_hierarchical_model(self,
                                y: np.ndarray,
                                X_media: np.ndarray,
                                geo_idx: np.ndarray) -> pm.Model:
        """Build hierarchical model across geographies.
        
        Args:
            y: Target variable (all geos concatenated)
            X_media: Media spend matrix
            geo_idx: Geography index for each observation
        """
        
        n_channels = X_media.shape[1]
        n_geos = len(self.geo_names)
        
        with pm.Model() as model:
            # Hyperpriors for media coefficients
            mu_beta = pm.HalfNormal('mu_beta', sigma=1, shape=n_channels)
            sigma_beta = pm.HalfNormal('sigma_beta', sigma=0.5, shape=n_channels)
            
            # Geo-specific media coefficients
            beta_media = pm.Normal('beta_media', 
                                  mu=mu_beta,
                                  sigma=sigma_beta,
                                  shape=(n_geos, n_channels))
            
            # Shared adstock parameters
            theta = pm.Beta('theta', alpha=2, beta=2, shape=n_channels)
            
            # Shared saturation parameters
            K = pm.HalfNormal('K', sigma=1, shape=n_channels)
            S = pm.HalfNormal('S', sigma=1, shape=n_channels)
            
            # Transform media (shared transformations)
            media_transformed = tt.zeros_like(X_media)
            for i in range(n_channels):
                adstocked = self._geometric_adstock_theano(X_media[:, i], theta[i])
                media_transformed = tt.set_subtensor(
                    media_transformed[:, i],
                    adstocked ** S[i] / (K[i] ** S[i] + adstocked ** S[i])
                )
            
            # Geo-specific intercepts
            intercept = pm.Normal('intercept', mu=y.mean(), sigma=y.std(), shape=n_geos)
            
            # Linear combination
            mu = intercept[geo_idx] + tt.sum(
                media_transformed * beta_media[geo_idx, :],
                axis=1
            )
            
            # Likelihood
            sigma = pm.HalfNormal('sigma', sigma=y.std())
            likelihood = pm.Normal('y', mu=mu, sigma=sigma, observed=y)
        
        self.model = model
        return model
    
    def _geometric_adstock_theano(self, x, theta):
        """Geometric adstock in Theano."""
        
        def adstock_step(x_t, adstock_prev, theta):
            return x_t + theta * adstock_prev
        
        adstock, _ = theano.scan(
            fn=adstock_step,
            sequences=x,
            outputs_info=tt.zeros(1),
            non_sequences=theta
        )
        
        return adstock.flatten()
    
    def fit(self, draws: int = 2000, tune: int = 1000, chains: int = 2):
        """Fit hierarchical model."""
        
        self.logger.info("Starting hierarchical MCMC sampling")
        
        with self.model:
            self.trace = pm.sample(
                draws=draws,
                tune=tune,
                chains=chains,
                target_accept=0.9,
                return_inferencedata=True
            )
        
        return self.trace
    
    def get_geo_roas(self) -> pd.DataFrame:
        """Get ROAS by geography."""
        
        beta_media = self.trace.posterior['beta_media'].mean(dim=['chain', 'draw']).values
        
        results = []
        for geo_idx, geo_name in enumerate(self.geo_names):
            for ch_idx, ch_name in enumerate(self.channel_names):
                results.append({
                    'geo': geo_name,
                    'channel': ch_name,
                    'coefficient': beta_media[geo_idx, ch_idx]
                })
        
        return pd.DataFrame(results)


print("Hierarchical MMM class ready")

## 5. Automated Model Selection and Validation

In [ ]:
class MMMValidator:
    """Automated validation for MMM models."""
    
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def time_series_cv(self, 
                      X: np.ndarray,
                      y: np.ndarray,
                      model_class,
                      n_splits: int = 5) -> Dict:
        """Time series cross-validation."""
        
        tscv = TimeSeriesSplit(n_splits=n_splits)
        scores = []
        
        for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
            self.logger.info(f"Fold {fold}/{n_splits}")
            
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            # Fit model
            model = model_class()
            model.fit(X_train, y_train)
            
            # Predict
            y_pred = model.predict(X_val)
            
            # Calculate metrics
            scores.append({
                'fold': fold,
                'r2': r2_score(y_val, y_pred),
                'mae': mean_absolute_error(y_val, y_pred),
                'rmse': np.sqrt(mean_squared_error(y_val, y_pred)),
                'mape': np.mean(np.abs((y_val - y_pred) / y_val)) * 100
            })
        
        return pd.DataFrame(scores)
    
    def check_multicollinearity(self, X: pd.DataFrame) -> pd.DataFrame:
        """Calculate VIF to check multicollinearity."""
        
        vif_data = []
        
        for i in range(X.shape[1]):
            vif = variance_inflation_factor(X.values, i)
            vif_data.append({
                'variable': X.columns[i],
                'vif': vif,
                'severity': 'High' if vif > 10 else ('Medium' if vif > 5 else 'Low')
            })
        
        return pd.DataFrame(vif_data).sort_values('vif', ascending=False)
    
    def residual_analysis(self, y_true: np.ndarray, y_pred: np.ndarray) -> Dict:
        """Analyze model residuals."""
        
        residuals = y_true - y_pred
        
        # Normality test
        _, p_value = stats.normaltest(residuals)
        
        # Autocorrelation
        acf_values = acf(residuals, nlags=20)
        
        # Homoscedasticity
        _, bp_p_value = stats.levene(y_pred, residuals)
        
        return {
            'mean_residual': residuals.mean(),
            'std_residual': residuals.std(),
            'normality_p_value': p_value,
            'is_normal': p_value > 0.05,
            'autocorrelation': acf_values,
            'homoscedasticity_p_value': bp_p_value
        }
    
    def compare_models(self, models: Dict, X: np.ndarray, y: np.ndarray) -> pd.DataFrame:
        """Compare multiple model specifications."""
        
        results = []
        
        # Split data
        split_idx = int(len(X) * 0.8)
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]
        
        for model_name, model in models.items():
            self.logger.info(f"Evaluating {model_name}")
            
            start_time = time.time()
            model.fit(X_train, y_train)
            fit_time = time.time() - start_time
            
            y_pred = model.predict(X_test)
            
            results.append({
                'model': model_name,
                'r2': r2_score(y_test, y_pred),
                'mae': mean_absolute_error(y_test, y_pred),
                'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
                'mape': np.mean(np.abs((y_test - y_pred) / y_test)) * 100,
                'fit_time': fit_time
            })
        
        return pd.DataFrame(results).sort_values('r2', ascending=False)


validator = MMMValidator()
print("MMM validator ready")

## 6. Budget Optimization with Constraints

In [ ]:
class BudgetOptimizer:
    """Optimize marketing budget allocation across channels."""
    
    def __init__(self, channel_names: List[str]):
        self.channel_names = channel_names
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def optimize_budget(self,
                       response_curves: Dict[str, callable],
                       total_budget: float,
                       min_spend: Optional[Dict[str, float]] = None,
                       max_spend: Optional[Dict[str, float]] = None) -> pd.DataFrame:
        """Optimize budget allocation to maximize total response.
        
        Args:
            response_curves: Dict mapping channel name to response function
            total_budget: Total budget to allocate
            min_spend: Minimum spend per channel
            max_spend: Maximum spend per channel
        """
        
        n_channels = len(self.channel_names)
        
        # Decision variables
        spend = cp.Variable(n_channels, nonneg=True)
        
        # Objective: maximize total response
        # Using piecewise linear approximation for response curves
        responses = []
        for i, channel in enumerate(self.channel_names):
            # Approximate response curve
            x_points = np.linspace(0, total_budget / n_channels * 2, 100)
            y_points = np.array([response_curves[channel](x) for x in x_points])
            
            # Linear interpolation (piecewise linear)
            response = cp.sum([
                (y_points[j+1] - y_points[j]) / (x_points[j+1] - x_points[j]) * 
                cp.maximum(0, cp.minimum(spend[i] - x_points[j], x_points[j+1] - x_points[j]))
                for j in range(len(x_points) - 1)
            ])
            responses.append(response)
        
        objective = cp.Maximize(cp.sum(responses))
        
        # Constraints
        constraints = [cp.sum(spend) <= total_budget]
        
        # Min/max spend constraints
        if min_spend:
            for i, channel in enumerate(self.channel_names):
                if channel in min_spend:
                    constraints.append(spend[i] >= min_spend[channel])
        
        if max_spend:
            for i, channel in enumerate(self.channel_names):
                if channel in max_spend:
                    constraints.append(spend[i] <= max_spend[channel])
        
        # Solve
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        if problem.status == 'optimal':
            optimal_spend = spend.value
            
            results = []
            for i, channel in enumerate(self.channel_names):
                results.append({
                    'channel': channel,
                    'optimal_spend': optimal_spend[i],
                    'expected_response': response_curves[channel](optimal_spend[i]),
                    'marginal_roas': self._marginal_roas(response_curves[channel], optimal_spend[i])
                })
            
            return pd.DataFrame(results)
        else:
            self.logger.error(f"Optimization failed: {problem.status}")
            return None
    
    def _marginal_roas(self, response_func: callable, spend: float, delta: float = 1.0) -> float:
        """Calculate marginal ROAS at given spend level."""
        return (response_func(spend + delta) - response_func(spend)) / delta
    
    def scenario_analysis(self,
                         response_curves: Dict[str, callable],
                         budget_scenarios: List[float]) -> pd.DataFrame:
        """Analyze optimal allocation across different budget scenarios."""
        
        results = []
        
        for budget in budget_scenarios:
            optimal = self.optimize_budget(response_curves, budget)
            
            if optimal is not None:
                total_response = optimal['expected_response'].sum()
                avg_roas = total_response / budget
                
                results.append({
                    'total_budget': budget,
                    'total_response': total_response,
                    'average_roas': avg_roas,
                    'allocation': optimal.to_dict('records')
                })
        
        return pd.DataFrame(results)


print("Budget optimizer ready")

## 7. Forecasting at Scale

In [ ]:
class MMMForecaster:
    """Forecast future performance using trained MMM."""
    
    def __init__(self, model):
        self.model = model
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def forecast_scenarios(self,
                          future_media: Dict[str, np.ndarray],
                          n_simulations: int = 1000) -> pd.DataFrame:
        """Generate forecasts for different media scenarios.
        
        Args:
            future_media: Dict mapping scenario name to media spend matrix
            n_simulations: Number of Monte Carlo simulations
        """
        
        forecasts = []
        
        for scenario_name, X_future in future_media.items():
            self.logger.info(f"Forecasting scenario: {scenario_name}")
            
            # Monte Carlo simulation from posterior
            predictions = []
            for _ in range(n_simulations):
                # Sample from posterior
                y_pred = self._sample_prediction(X_future)
                predictions.append(y_pred)
            
            predictions = np.array(predictions)
            
            # Calculate statistics
            forecasts.append({
                'scenario': scenario_name,
                'mean_forecast': predictions.mean(axis=0),
                'lower_95': np.percentile(predictions, 2.5, axis=0),
                'upper_95': np.percentile(predictions, 97.5, axis=0),
                'std': predictions.std(axis=0)
            })
        
        return forecasts
    
    def _sample_prediction(self, X: np.ndarray) -> np.ndarray:
        """Sample prediction from posterior distribution."""
        # Implementation depends on model type
        # For Bayesian models, sample from trace
        # For frequentist models, use bootstrap
        pass
    
    def what_if_analysis(self,
                        baseline_spend: Dict[str, float],
                        channel_to_vary: str,
                        spend_range: Tuple[float, float],
                        n_points: int = 50) -> pd.DataFrame:
        """Analyze impact of varying spend on one channel."""
        
        results = []
        
        spend_values = np.linspace(spend_range[0], spend_range[1], n_points)
        
        for spend in spend_values:
            # Create scenario
            scenario_spend = baseline_spend.copy()
            scenario_spend[channel_to_vary] = spend
            
            # Predict
            X_scenario = self._create_feature_matrix(scenario_spend)
            y_pred = self.model.predict(X_scenario)
            
            results.append({
                'channel': channel_to_vary,
                'spend': spend,
                'predicted_sales': y_pred.mean(),
                'incremental_sales': y_pred.mean() - baseline_pred
            })
        
        return pd.DataFrame(results)
    
    def _create_feature_matrix(self, spend_dict: Dict[str, float]) -> np.ndarray:
        """Create feature matrix from spend dictionary."""
        # Implementation depends on model features
        pass


print("MMM forecaster ready")

## 8. Real-World Project: Enterprise MMM Platform

In [ ]:
class EnterpriseMMM:
    """End-to-end enterprise MMM platform."""
    
    def __init__(self, config: Dict):
        self.config = config
        self.logger = logging.getLogger(self.__class__.__name__)
        self.models = {}
        self.results = {}
    
    def run_full_pipeline(self,
                         data: pd.DataFrame,
                         target_col: str,
                         media_cols: List[str],
                         control_cols: Optional[List[str]] = None) -> Dict:
        """Execute complete MMM pipeline."""
        
        self.logger.info("Starting enterprise MMM pipeline")
        
        # 1. Data preparation
        self.logger.info("Step 1: Data preparation")
        X_media, X_control, y = self._prepare_data(data, target_col, media_cols, control_cols)
        
        # 2. Transform features (adstock + saturation)
        self.logger.info("Step 2: Feature engineering")
        X_transformed = self._engineer_features(X_media)
        
        # 3. Train multiple models
        self.logger.info("Step 3: Model training")
        self.models = self._train_models(X_transformed, y, X_control)
        
        # 4. Model validation
        self.logger.info("Step 4: Model validation")
        validation_results = self._validate_models(X_transformed, y)
        
        # 5. Select best model
        self.logger.info("Step 5: Model selection")
        best_model = self._select_best_model(validation_results)
        
        # 6. Calculate ROI and attribution
        self.logger.info("Step 6: Attribution analysis")
        attribution = self._calculate_attribution(best_model, X_media, media_cols)
        
        # 7. Budget optimization
        self.logger.info("Step 7: Budget optimization")
        optimal_budget = self._optimize_budget(best_model, media_cols)
        
        # 8. Generate forecasts
        self.logger.info("Step 8: Forecasting")
        forecasts = self._generate_forecasts(best_model, media_cols)
        
        # 9. Create reports
        self.logger.info("Step 9: Report generation")
        reports = self._generate_reports(attribution, optimal_budget, forecasts)
        
        return {
            'best_model': best_model,
            'validation': validation_results,
            'attribution': attribution,
            'optimal_budget': optimal_budget,
            'forecasts': forecasts,
            'reports': reports
        }
    
    def _prepare_data(self, data: pd.DataFrame, target_col: str,
                     media_cols: List[str], control_cols: Optional[List[str]]) -> Tuple:
        """Prepare and clean data."""
        
        # Extract arrays
        y = data[target_col].values
        X_media = data[media_cols].values
        X_control = data[control_cols].values if control_cols else None
        
        # Handle missing values
        X_media = np.nan_to_num(X_media, 0)
        if X_control is not None:
            X_control = np.nan_to_num(X_control, 0)
        
        return X_media, X_control, y
    
    def _engineer_features(self, X_media: np.ndarray) -> np.ndarray:
        """Apply adstock and saturation transformations."""
        
        n_channels = X_media.shape[1]
        X_transformed = np.zeros_like(X_media, dtype=float)
        
        # Apply transformations to each channel
        for i in range(n_channels):
            # Use default parameters or optimize
            theta = 0.5  # Can be optimized
            K = X_media[:, i].mean()
            S = 1.0
            
            X_transformed[:, i] = AdstockTransformations.transform_channel(
                X_media[:, i], theta, K, S
            )
        
        return X_transformed
    
    def _train_models(self, X: np.ndarray, y: np.ndarray, 
                     X_control: Optional[np.ndarray]) -> Dict:
        """Train multiple model specifications."""
        
        models = {}
        
        # Combine features
        if X_control is not None:
            X_full = np.hstack([X, X_control])
        else:
            X_full = X
        
        # Train different models
        models['ridge'] = Ridge(alpha=1.0).fit(X_full, y)
        models['lasso'] = Lasso(alpha=0.1).fit(X_full, y)
        models['elastic_net'] = ElasticNet(alpha=0.1, l1_ratio=0.5).fit(X_full, y)
        
        return models
    
    def _validate_models(self, X: np.ndarray, y: np.ndarray) -> pd.DataFrame:
        """Validate all trained models."""
        
        validator = MMMValidator()
        return validator.compare_models(self.models, X, y)
    
    def _select_best_model(self, validation_results: pd.DataFrame):
        """Select best performing model."""
        best_model_name = validation_results.iloc[0]['model']
        return self.models[best_model_name]
    
    def _calculate_attribution(self, model, X_media: np.ndarray, 
                              media_cols: List[str]) -> pd.DataFrame:
        """Calculate channel attribution."""
        
        # Get coefficients
        coefficients = model.coef_[:len(media_cols)]
        
        # Calculate contributions
        contributions = X_media * coefficients
        
        return pd.DataFrame({
            'channel': media_cols,
            'total_contribution': contributions.sum(axis=0),
            'avg_contribution': contributions.mean(axis=0),
            'coefficient': coefficients
        }).sort_values('total_contribution', ascending=False)
    
    def _optimize_budget(self, model, media_cols: List[str]) -> pd.DataFrame:
        """Optimize budget allocation."""
        
        # Create response curves from model
        # This is simplified - real implementation would be more complex
        optimizer = BudgetOptimizer(media_cols)
        
        # Placeholder - would use actual model predictions
        return pd.DataFrame({
            'channel': media_cols,
            'optimal_spend': [100000] * len(media_cols)
        })
    
    def _generate_forecasts(self, model, media_cols: List[str]) -> Dict:
        """Generate future forecasts."""
        
        # Placeholder
        return {'forecast_periods': 12, 'forecasts': []}
    
    def _generate_reports(self, attribution: pd.DataFrame,
                         optimal_budget: pd.DataFrame,
                         forecasts: Dict) -> Dict:
        """Generate executive reports."""
        
        summary = f"""
        MARKETING MIX MODEL SUMMARY
        ==========================
        
        Top Performing Channels:
        {attribution.head(3).to_string()}
        
        Recommended Budget Allocation:
        {optimal_budget.to_string()}
        """
        
        return {
            'executive_summary': summary,
            'detailed_attribution': attribution,
            'budget_recommendations': optimal_budget
        }


# Example usage
# mmm = EnterpriseMMM({'region': 'US'})
# results = mmm.run_full_pipeline(
#     data=marketing_data,
#     target_col='sales',
#     media_cols=['tv', 'digital', 'print'],
#     control_cols=['seasonality', 'promotions']
# )

print("Enterprise MMM platform ready")

## 9. Exercises

### Exercise 1: Build Bayesian MMM
1. Load marketing dataset with 5+ channels
2. Apply adstock and saturation transformations
3. Build and train Bayesian MMM with PyMC3
4. Analyze posterior distributions
5. Calculate ROAS by channel

### Exercise 2: Hierarchical Modeling
1. Prepare multi-geo dataset
2. Build hierarchical MMM
3. Compare performance across geographies
4. Identify geo-specific vs. shared effects

### Exercise 3: Budget Optimization
1. Extract response curves from trained model
2. Set up optimization problem with constraints
3. Find optimal budget allocation
4. Perform sensitivity analysis
5. Generate scenario comparisons

### Exercise 4: Production Deployment
1. Build automated MMM pipeline
2. Implement model versioning
3. Create API endpoints for predictions
4. Set up monitoring and alerting
5. Generate automated reports

## Resources

### Documentation
- [PyMC3 Documentation](https://docs.pymc.io/)
- [ArviZ - Exploratory Analysis of Bayesian Models](https://arviz-devs.github.io/arviz/)
- [CVXPY - Convex Optimization](https://www.cvxpy.org/)
- [Meta Robyn MMM](https://github.com/facebookexperimental/Robyn)

### Papers & Resources
- Jin, Y., et al. (2017). Bayesian Methods for Media Mix Modeling
- Shapiro, C. (2018). Marketing Attribution Models
- Google: Meridian MMM
- Chan & Perry: Challenges in Marketing Mix Modeling

### Tools
- Robyn (Meta): Open-source MMM
- LightweightMMM (Google): Bayesian MMM in JAX
- PyMC-Marketing: Marketing analytics with PyMC
- Orbit: Bayesian time series framework